In [8]:
# pip install torchvision

In [30]:
# ===========================
# 1. Imports
# ===========================
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [31]:
# ===========================
# 2. Reproducibility Setup
# ===========================
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [32]:
# ===========================
# 3. DOWNLOAD & UNZIP DATA
# ===========================
#!wget -q https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
#!unzip -q data.zip
#print("Data folders:", os.listdir("data"))

In [33]:
# ===========================
# 4. Data Transforms
# ===========================
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

test_transforms = train_transforms

In [34]:
# ===========================
# 5. DATASET AND DATALOADER
# ===========================
train_dataset = datasets.ImageFolder("data/train", transform=train_transforms)
test_dataset  = datasets.ImageFolder("data/test",  transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
validation_loader = DataLoader(test_dataset, batch_size=20, shuffle=False)

print("Classes:", train_dataset.classes)
print("Train samples:", len(train_dataset), "Test samples:", len(test_dataset))

Classes: ['curly', 'straight']
Train samples: 800 Test samples: 201


In [35]:
# ===========================
# 6. Model Definition
# ===========================
class HairTypeCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=0)
        self.fc1 = nn.Linear(32*99*99, 64)
        self.fc2 = nn.Linear(64, 1)  # Binary classification

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # NO sigmoid here (BCEWithLogitsLoss)
        return x

model = HairTypeCNN().to(device)
print(model)

HairTypeCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=313632, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)


In [36]:
# ===========================
# 7. LOSS & OPTIMIZER
# ===========================
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

In [37]:
# ===========================
# 8. Training Loop
# ===========================
num_epochs = 10
history = {'loss': [], 'acc': [], 'val_loss': [], 'val_acc': []}

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = train_loss / len(train_loader.dataset)
    epoch_acc = correct / total
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    # Validation
    model.eval()
    val_loss = 0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.float().unsqueeze(1).to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_loss / len(test_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6462, Acc: 0.6362, Val Loss: 0.6032, Val Acc: 0.6517
Epoch 2/10, Loss: 0.5475, Acc: 0.7100, Val Loss: 0.7251, Val Acc: 0.6318
Epoch 3/10, Loss: 0.5533, Acc: 0.7250, Val Loss: 0.5991, Val Acc: 0.6716
Epoch 4/10, Loss: 0.4802, Acc: 0.7712, Val Loss: 0.6033, Val Acc: 0.6567
Epoch 5/10, Loss: 0.4334, Acc: 0.8025, Val Loss: 0.6196, Val Acc: 0.6766
Epoch 6/10, Loss: 0.3740, Acc: 0.8325, Val Loss: 0.7371, Val Acc: 0.6766
Epoch 7/10, Loss: 0.2721, Acc: 0.8838, Val Loss: 0.9223, Val Acc: 0.6418
Epoch 8/10, Loss: 0.2478, Acc: 0.9000, Val Loss: 0.7294, Val Acc: 0.7214
Epoch 9/10, Loss: 0.2075, Acc: 0.9200, Val Loss: 0.7523, Val Acc: 0.7015
Epoch 10/10, Loss: 0.1494, Acc: 0.9450, Val Loss: 0.7894, Val Acc: 0.7015


In [38]:
# ===========================
# 9. STATISTICS (Q3 & Q4)
# ===========================
median_acc = np.median(history['acc'])
std_loss = np.std(history['loss'])
print("Median training accuracy:", median_acc)
print("STD of training loss:", std_loss)

Median training accuracy: 0.8175
STD of training loss: 0.15896371677643137


In [39]:
# ===========================
# 10. DATA AUGMENTATION
# ===========================
augmented_train_transforms = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9,1.0), ratio=(0.9,1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

aug_train_dataset = datasets.ImageFolder("data/train", transform=augmented_train_transforms)
aug_train_loader = DataLoader(aug_train_dataset, batch_size=20, shuffle=True)

In [ ]:
# ===========================
# 11. CONTINUE TRAINING WITH AUGMENTATION (Q5 & Q6)
# ===========================
num_epochs_aug = 10
aug_history = {'loss': [], 'acc': [], 'val_loss': [], 'val_acc': []}

for epoch in range(num_epochs_aug):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for images, labels in aug_train_loader:
        images, labels = images.to(device), labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = train_loss / len(aug_train_dataset)
    epoch_acc = correct / total
    aug_history['loss'].append(epoch_loss)
    aug_history['acc'].append(epoch_acc)

    # Validation
    model.eval()
    val_loss = 0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.float().unsqueeze(1).to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_loss / len(test_dataset)
    val_epoch_acc = correct_val / total_val
    aug_history['val_loss'].append(val_epoch_loss)
    aug_history['val_acc'].append(val_epoch_acc)

    print(f"Aug Epoch {epoch+1}/10 — Train Loss: {epoch_loss:.4f} — Val Loss: {val_epoch_loss:.4f} — Val Acc: {val_epoch_acc:.4f}")

Aug Epoch 1/10 — Train Loss: 0.7418 — Val Loss: 0.6577 — Val Acc: 0.6816
Aug Epoch 2/10 — Train Loss: 0.5797 — Val Loss: 0.7310 — Val Acc: 0.6766
Aug Epoch 3/10 — Train Loss: 0.5754 — Val Loss: 0.5767 — Val Acc: 0.6965
Aug Epoch 4/10 — Train Loss: 0.5522 — Val Loss: 0.5689 — Val Acc: 0.7015
Aug Epoch 5/10 — Train Loss: 0.5592 — Val Loss: 0.6367 — Val Acc: 0.6617
Aug Epoch 6/10 — Train Loss: 0.5235 — Val Loss: 0.5813 — Val Acc: 0.7065
Aug Epoch 7/10 — Train Loss: 0.5243 — Val Loss: 0.7844 — Val Acc: 0.6119
Aug Epoch 8/10 — Train Loss: 0.5057 — Val Loss: 0.9460 — Val Acc: 0.6269
Aug Epoch 9/10 — Train Loss: 0.5030 — Val Loss: 0.5460 — Val Acc: 0.7164
Aug Epoch 10/10 — Train Loss: 0.4755 — Val Loss: 0.5426 — Val Acc: 0.7562
Mean test loss (augmented): 0.6571219857504118
Average test accuracy last 5 epochs (augmented): 0.6835820895522388


In [41]:
# ===========================
# 12. AUGMENTATION STATS (Q5 & Q6)
# ===========================
mean_test_loss = np.mean(aug_history['val_loss'])
avg_last5_acc = np.mean(aug_history['val_acc'][5:10])
print("Mean test loss (augmented):", mean_test_loss)
print("Average test accuracy last 5 epochs (augmented):", avg_last5_acc)

Mean test loss (augmented): 0.6571219857504118
Average test accuracy last 5 epochs (augmented): 0.6835820895522388
